In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Project_1')

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
from pathlib import Path
import tensorflow as tf

# Update the image directory path
img_directory = "Train"
train = pd.read_csv('train.csv')  # adjust filename and path as necessary

# Load images without considering the directory-generated labels
image_size = (1024, 1024)
batch_size = 4

# This loads the images but ignores the labels
image_dataset = tf.keras.utils.image_dataset_from_directory(
    img_directory,
    label_mode=None,  # Ignore directory-based labels
    image_size=image_size,
    batch_size=batch_size
)

# Extract file paths from the dataset
image_file_paths = image_dataset.file_paths

# Extract only file ID without extension from file paths
image_hids = [Path(image_file_path).stem for image_file_path in image_file_paths]
image_hids_int = [int(house_id) for house_id in image_hids]

labels = [train.loc[train['House ID'] == houseid, 'Price'].values for houseid in image_hids_int]

# Convert the list of labels into a TensorFlow dataset
labels_dataset = tf.data.Dataset.from_tensor_slices(labels).batch(batch_size)

# Combine the original image dataset with the labels dataset
final_dataset = tf.data.Dataset.zip((image_dataset, labels_dataset))

Found 500 files belonging to 1 classes.


In [ ]:
num_samples = sum(1 for _ in final_dataset)
val_size = int(0.2 * num_samples)  # 20% of the dataset for validation

# Split the dataset into training and validation sets
val_dataset = final_dataset.take(val_size)
train_dataset = final_dataset.skip(val_size)

In [ ]:
img_height, img_width = 256, 256  # Define your desired image size
train_dataset = train_dataset.map(lambda x, y: (tf.image.resize(x, (img_height, img_width)), y))

In [ ]:
val_dataset = val_dataset.map(lambda x, y: (tf.image.resize(x, (img_height, img_width)), y))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(5).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

normalization_layer = tf.keras.layers.Rescaling(
    scale=1./255, offset=0.0)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                            patience=5,
                                            verbose=1,
                                            factor=0.2,
                                            min_lr=0.00000001)
model19 = tf.keras.Sequential([
    keras.Input(shape=((256,256,3))),
    normalization_layer,
    tf.keras.layers.Conv2D(96, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(240, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(3),
    tf.keras.layers.Conv2D(192, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])



In [ ]:
opt= keras.optimizers.Adam(learning_rate=0.001)

model19.compile(optimizer=opt,
              loss='mape')

In [ ]:
history = model19.fit(train_dataset, epochs=1000000, validation_data=val_dataset,callbacks=[stop_early])


Epoch 1/1000000
100/100 [==============================] - 5s 35ms/step - loss: 76.7566 - val_loss: 67.8213
Epoch 2/1000000
100/100 [==============================] - 3s 31ms/step - loss: 74.0675 - val_loss: 72.8613
Epoch 3/1000000
100/100 [==============================] - 3s 31ms/step - loss: 72.5948 - val_loss: 78.8693
Epoch 4/1000000
100/100 [==============================] - 3s 34ms/step - loss: 75.1027 - val_loss: 73.6105
Epoch 5/1000000
100/100 [==============================] - 3s 31ms/step - loss: 74.4844 - val_loss: 67.9874
Epoch 6/1000000
100/100 [==============================] - 3s 31ms/step - loss: 72.3110 - val_loss: 70.2428
Epoch 7/1000000
100/100 [==============================] - 3s 33ms/step - loss: 73.1942 - val_loss: 69.0756
Epoch 8/1000000
100/100 [==============================] - 3s 31ms/step - loss: 73.6545 - val_loss: 79.1581
Epoch 9/1000000
100/100 [==============================] - 3s 32ms/step - loss: 72.4749 - val_loss: 73.0717
Epoch 10/1000000
100/100 [==

In [ ]:
from tensorflow.keras.regularizers import l1, l2, l1_l2
from keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

normalization_layer = tf.keras.layers.Rescaling(
    scale=1./255, offset=0.0)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                            patience=3,
                                            verbose=1,
                                            factor=0.2,
                                            min_lr=0.00000001)

def model_builder2(hp):
    model = keras.Sequential()
    model.add(keras.Input(shape=((256,256,3))))
    model.add(normalization_layer)
    model.add(layers.Conv2D(hp.Int('filters_1', min_value=32, max_value=128, step=16),
                             hp.Choice('kernel_size_1', values=[3, 5]),
                             activation='relu'))
    model.add(layers.MaxPooling2D(hp.Choice('pooling_size_1', values=[2, 3])))

    model.add(layers.Conv2D(hp.Int('filters_2', min_value=64, max_value=256, step=16),
                             hp.Choice('kernel_size_2', values=[3, 5]),
                             activation='relu'))
    model.add(layers.MaxPooling2D(hp.Choice('pooling_size_2', values=[2, 3])))

    model.add(layers.Conv2D(hp.Int('filters_3', min_value=128, max_value=512, step=32),
                             hp.Choice('kernel_size_3', values=[3, 5]),
                             activation='relu'))
    model.add(layers.MaxPooling2D(hp.Choice('pooling_size_3', values=[2, 3])))
    model.add(layers.Flatten())

    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    reg_choice = hp.Choice('kernel_regularizer', values=['l1', 'l2', 'l1l2', 'none'])
    if reg_choice == 'l1':
        model.add(keras.layers.Dense(units=hp_units, activation='relu', kernel_regularizer=l1(hp.Float('l1_regularization', min_value=1e-5, max_value=0.01, sampling='log'))))
    elif reg_choice == 'l2':
        model.add(keras.layers.Dense(units=hp_units, activation='relu', kernel_regularizer=l2(hp.Float('l2_regularization', min_value=1e-5, max_value=0.01, sampling='log'))))
    elif reg_choice == 'l1l2':
        model.add(keras.layers.Dense(units=hp_units, activation='relu', kernel_regularizer=l1_l2(hp.Float('l1l2_l1', min_value=1e-5, max_value=0.01, sampling='log'), hp.Float('l1l2_l2', min_value=0.0, max_value=0.01, sampling='log'))))
    else:
        model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dropout(rate=hp.Float('dropout_rate',
                                               min_value=0.0,
                                               max_value=0.5,
                                               step=0.1)))
    model.add(keras.layers.Dense(1))


    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mape')

    return model

In [ ]:
#!pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.6 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt
tuner3 = kt.Hyperband(model_builder2,
                     objective='val_loss',
                     max_epochs=50,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_ktyo1',
                     max_consecutive_failed_trials=10)

In [ ]:
tuner3.search(train_dataset, epochs=50, validation_data=val_dataset,callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps2=tuner3.get_best_hyperparameters(num_trials=1)[0]




Trial 69 Complete [00h 00m 58s]
val_loss: 64.51725769042969

Best val_loss So Far: 63.02580261230469
Total elapsed time: 00h 32m 46s


NameError: name 'best_hps' is not defined

In [ ]:
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps2.get('units')} and the optimal learning rate for the optimizer
is {best_hps2.get('learning_rate')}.

Other hyperparameters:
- Filters 1: {best_hps2.get('filters_1')}
- Kernel size 1: {best_hps2.get('kernel_size_1')}
- Pooling size 1: {best_hps2.get('pooling_size_1')}
- Filters 2: {best_hps2.get('filters_2')}
- Kernel size 2: {best_hps2.get('kernel_size_2')}
- Pooling size 2: {best_hps2.get('pooling_size_2')}
- Filters 3: {best_hps2.get('filters_3')}
- Kernel size 3: {best_hps2.get('kernel_size_3')}
- Pooling size 3: {best_hps2.get('pooling_size_3')}
- Dropout rate: {best_hps2.get('dropout_rate')}
- Kernel regularization: {best_hps2.get('kernel_regularizer')}
""")


NameError: name 'best_hps2' is not defined

In [ ]:
from tensorflow.keras.regularizers import l1, l2, l1_l2
from keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
normalization_layer = tf.keras.layers.Rescaling(
    scale=1./255, offset=0.0)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                            patience=3,
                                            verbose=1,
                                            factor=0.2,
                                            min_lr=0.00000001)

def model_builder3(hp):
    model = keras.Sequential()
    model.add(keras.Input(shape=((256,256,3))))
    model.add(normalization_layer)
    model.add(layers.Conv2D(hp.Int('filters_1', min_value=32, max_value=128, step=16),
                             hp.Choice('kernel_size_1', values=[3, 5])))
    model.add(layers.BatchNormalization())  # Add BatchNormalization here
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(hp.Choice('pooling_size_1', values=[2, 3])))

    model.add(layers.Conv2D(hp.Int('filters_2', min_value=64, max_value=256, step=16),
                             hp.Choice('kernel_size_2', values=[3, 5])))
    model.add(layers.BatchNormalization())  # Add BatchNormalization here
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(hp.Choice('pooling_size_2', values=[2, 3])))

    model.add(layers.Conv2D(hp.Int('filters_3', min_value=128, max_value=512, step=32),
                             hp.Choice('kernel_size_3', values=[3, 5])))
    model.add(layers.BatchNormalization())  # Add BatchNormalization here
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(hp.Choice('pooling_size_3', values=[2, 3])))
    model.add(layers.Flatten())

    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    reg_choice = hp.Choice('kernel_regularizer', values=['l1', 'l2', 'l1l2', 'none'])
    if reg_choice == 'l1':
        model.add(keras.layers.Dense(units=hp_units, activation='relu', kernel_regularizer=l1(hp.Float('l1_regularization', min_value=1e-5, max_value=0.01, sampling='log'))))
    elif reg_choice == 'l2':
        model.add(keras.layers.Dense(units=hp_units, activation='relu', kernel_regularizer=l2(hp.Float('l2_regularization', min_value=1e-5, max_value=0.01, sampling='log'))))
    elif reg_choice == 'l1l2':
        model.add(keras.layers.Dense(units=hp_units, activation='relu', kernel_regularizer=l1_l2(hp.Float('l1l2_l1', min_value=1e-5, max_value=0.01, sampling='log'), hp.Float('l1l2_l2', min_value=0.0, max_value=0.01, sampling='log'))))
    else:
        model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dropout(rate=hp.Float('dropout_rate',
                                               min_value=0.0,
                                               max_value=0.5,
                                               step=0.1)))
    model.add(keras.layers.Dense(1))


    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mape')

    return model

In [ ]:
import keras_tuner as kt
tuner4 = kt.Hyperband(model_builder3,
                     objective='val_loss',
                     max_epochs=50,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_ktyo13',
                     max_consecutive_failed_trials=10)

In [ ]:
tuner4.search(train_dataset, epochs=50, validation_data=val_dataset,callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps2=tuner4.get_best_hyperparameters(num_trials=1)[0]



Trial 63 Complete [00h 00m 01s]

Best val_loss So Far: 64.68605041503906
Total elapsed time: 00h 22m 23s

Search: Running Trial #64

Value             |Best Value So Far |Hyperparameter
80                |64                |filters_1
5                 |3                 |kernel_size_1
3                 |2                 |pooling_size_1
160               |96                |filters_2
5                 |5                 |kernel_size_2
3                 |2                 |pooling_size_2
416               |192               |filters_3
5                 |3                 |kernel_size_3
2                 |2                 |pooling_size_3
256               |192               |units
l1l2              |none              |kernel_regularizer
4.5214e-05        |2.8339e-05        |l1_regularization
0.1               |0.2               |dropout_rate
0.01              |0.001             |learning_rate
0.00062424        |0.00058335        |l1l2_l1
0.00011431        |0.00042593        |l2_regulari

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 232, in _build_and_fit_model
    model = self._try_build(hp)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 164, in _t

RuntimeError: Number of consecutive failures exceeded the limit of 10.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 232, in _build_and_fit_model
    model = self._try_build(hp)
  File "/usr/local/lib/pyt

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2

# Load pre-trained MobileNetV2 model without top classification layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom regression layers
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(1)(x)  # Output layer for regression

# Create the transfer learning model
transfer_model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
transfer_model.compile(optimizer=opt, loss='mape')

# Print model summary
transfer_model.summary()

# Train the model
history = transfer_model.fit(train_dataset, epochs=1000000, validation_data=val_dataset, callbacks=[stop_early])


9406464/9406464 [==============================] - 2s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 128, 128, 32)         864       ['input_6[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 128, 128, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (N